<a href="https://colab.research.google.com/github/YaninaK/Nornickel-fm/blob/b1/notebooks/01_Baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Флотомашина
## 2. Подготовка данных

In [1]:
initiate = False
if initiate:
  !git init -q
  !git clone -b b1  https://github.com/YaninaK/Nornickel-fm.git -q

  from google.colab import drive
  drive.mount('/content/drive')
  !cp /content/drive/MyDrive/ML_projects/10_ts_optimisation/data/01_raw/* -r /content/Nornickel-fm/data/01_raw/

%cd /content/Nornickel-fm/

/content/Nornickel-fm


In [2]:
import os
import sys

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "src", "nornickel_fm"))

In [3]:
import joblib
import tempfile

import numpy as np
import pandas as pd
import tensorflow as tf

from data.make_dataset import load_data
from features.optimisation_ore import generate_optimal_features_ore
from features.optimisation_cu import generate_optimal_features_cu
from features.optimisation_ni import generate_optimal_features_ni

from features.model_input import make_dataset_ore, make_dataset_cu, make_dataset_ni
from models.model_fm_1 import get_model_fm_1
from models.model_fm_2_3 import get_model_fm_2_3
from models.model_fm_4_5_6 import get_model_fm_4_5_6

from models.tf_dataset_fm_1 import get_tf_dataset_fm_1
from models.tf_dataset_fm_2_3 import get_tf_dataset_fm_2_3
from models.tf_dataset_fm_4_5_6 import get_tf_dataset_fm_4_5_6

## 1. Data ingestion

In [4]:
df = load_data()

## 2. FM 1

### 2.1 Подготовка данных

In [5]:
df = generate_optimal_features_ore(df)
X_fm_1, y_fm_1 = make_dataset_ore(df)

print(f"X_fm1.shape = {X_fm_1.shape}\n")
X_fm_1.sample(2)

X_fm1.shape = (15299, 7)



,Cu_oreth,Ni_oreth,Ore_mass,Mass_1,Dens_1,Cu_1C,Ni_1C
10509,2.7740,1.7152,1393.5,1453.496948,1.428795,4.7991,NaN
16156,2.2021,1.5111,1333.0,1375.939209,1.406291,5.0084,NaN


In [6]:
X_fm_1.describe()

,Cu_oreth,Ni_oreth,Ore_mass,Mass_1,Dens_1,Cu_1C,Ni_1C
count,14209.000000,14209.000000,15299.000000,15299.000000,15299.000000,15299.000000,6630.000000
mean,2.683563,1.670827,1259.220995,1413.237297,1.413750,4.915226,3.101008
std,0.279562,0.163206,202.303914,116.296956,0.033451,0.273392,0.254406
min,1.697700,1.213300,0.000000,0.000000,0.000000,4.093300,2.257900
25%,2.501300,1.559100,1158.500000,1349.594849,1.395144,4.708950,2.923525
50%,2.667300,1.673000,1289.500000,1421.064819,1.415327,4.889200,3.095950
75%,2.859100,1.778500,1396.500000,1488.979675,1.434485,5.071350,3.265500
max,3.872500,2.617300,2000.000000,2002.289673,1.684976,7.963700,4.540800


In [7]:
print(f"y_fm_1.shape = {y_fm_1.shape}\n")
y_fm_1.sample(2)

y_fm_1.shape = (15299, 4)



,Ni_1C_min,Ni_1C_max,Cu_1C_min,Cu_1C_max
21284,2.5,3.3,4.7,5.2
30108,2.5,3.2,4.6,4.6


In [8]:
y_fm_1.describe()

,Ni_1C_min,Ni_1C_max,Cu_1C_min,Cu_1C_max
count,15299.000000,15299.000000,15299.000000,15299.000000
mean,2.667488,3.229237,4.602915,5.008001
std,0.142156,0.115044,0.193250,0.383531
min,2.500000,3.000000,4.000000,4.000000
25%,2.500000,3.200000,4.400000,4.600000
50%,2.700000,3.200000,4.600000,5.200000
75%,2.800000,3.300000,4.700000,5.200000
max,2.800000,3.400000,4.900000,5.600000


### 2.2. Модель

In [9]:
model_fm_1 = get_model_fm_1(X_fm_1)

In [10]:
model_fm_1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Cu_oreth (InputLayer)     │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Ni_oreth (InputLayer)     │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Ore_mass (InputLayer)     │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Mass_1 (InputLayer)       │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Dens_1 (InputLayer)       │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Cu_1C (InputLayer)        │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Ni_1C (InputLayer)        │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Cu_oreth             │ (None, 1)              │              3 │ Cu_oreth[0][0]         │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Ni_oreth             │ (None, 1)              │              3 │ Ni_oreth[0][0]         │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Ore_mass             │ (None, 1)              │              3 │ Ore_mass[0][0]         │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Mass_1               │ (None, 1)              │              3 │ Mass_1[0][0]           │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Dens_1               │ (None, 1)              │              3 │ Dens_1[0][0]           │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Cu_1C                │ (None, 1)              │              3 │ Cu_1C[0][0]            │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Ni_1C                │ (None, 1)              │              3 │ Ni_1C[0][0]            │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ all_features              │ (None, 7)              │              0 │ norm_Cu_oreth[0][0],   │
│ (Concatenate)             │                        │                │ norm_Ni_oreth[0][0],   │
│                           │                        │                │ norm_Ore_mass[0][0],   │
│                      

 Total params: 873 (3.44 KB)

 Trainable params: 852 (3.33 KB)

 Non-trainable params: 21 (112.00 B)

### 2.3. Dataset

In [11]:
df1 = pd.concat([X_fm_1, y_fm_1], axis=1).reset_index()
N = int(len(df1)*0.15)

valid_ind= np.random.choice(len(df1), N, replace=False)
train_ind = list(set(df1.index) - set(valid_ind))

ds_train_fm_1 = get_tf_dataset_fm_1(df1.loc[train_ind, :])
ds_valid_fm_1 = get_tf_dataset_fm_1(df1.loc[valid_ind, :])

In [12]:
ds_train_fm_1.element_spec

({'Cu_oreth': TensorSpec(shape=(1,), dtype=tf.float64, name='Cu_oreth'),
  'Ni_oreth': TensorSpec(shape=(1,), dtype=tf.float64, name='Ni_oreth'),
  'Ore_mass': TensorSpec(shape=(1,), dtype=tf.float64, name='Ore_mass'),
  'Mass_1': TensorSpec(shape=(1,), dtype=tf.float64, name='Mass_1'),
  'Dens_1': TensorSpec(shape=(1,), dtype=tf.float64, name='Dens_1'),
  'Cu_1C': TensorSpec(shape=(1,), dtype=tf.float64, name='Cu_1C'),
  'Ni_1C': TensorSpec(shape=(1,), dtype=tf.float64, name='Ni_1C')},
 TensorSpec(shape=(4,), dtype=tf.float64, name='output'))

In [13]:
X, y = next(iter(ds_train_fm_1.take(1)))
model_fm_1(X)

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=
array([[1.5149987e+17, 0.0000000e+00, 1.2487140e+17, 0.0000000e+00]],
      dtype=float32)>

### 2.4. Обучение модели

In [14]:
METRICS = [
    tf.keras.metrics.MeanAbsoluteError(name="mae"),
    tf.keras.metrics.MeanAbsolutePercentageError(name="mape"),
    tf.keras.metrics.MeanSquaredError(name="mse"),
    tf.keras.metrics.RootMeanSquaredError(name="rmse")
]

In [15]:
i = 1
model_path = f"models/model_fm_1{i}.keras"
history_path = f"models/model_fm_1_training_history_v{i}.joblib"

disk_path = "/content/drive/MyDrive/ML_projects/10_ts_optimisation/"
checkpoints_path = disk_path + f'data/06_model_output/initial_weights_{i}.weights.h5'

BATCH_SIZE = 128

regenerate = True
if regenerate:
    initial_weights = os.path.join(tempfile.mkdtemp(), checkpoints_path)

    model_fm_1 = get_model_fm_1(X_fm_1)
    model_fm_1.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.Huber(),
        metrics=METRICS
    )
    #model_fm_1.load_weights(initial_weights)

    n_epochs = 2
    steps_per_epoch= 2
    batch_size = BATCH_SIZE
    reduce_lr = tf.keras.callbacks.LearningRateScheduler(
        lambda epoch: 3e-2 * 0.99**epoch
    )
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=30,
        min_delta=1e-06,
        verbose=1,
        mode="min",
        restore_best_weights=True,
    )

    model_fm_1_history = model_fm_1.fit(
        ds_train_fm_1.batch(batch_size),
        epochs=n_epochs,
        steps_per_epoch=steps_per_epoch,
        validation_data=ds_valid_fm_1.batch(batch_size),
        batch_size=batch_size,
        verbose=1,
        callbacks=[reduce_lr, early_stopping],
        shuffle=True,
    )
    model_fm_1.save(model_path)
    joblib.dump(model_fm_1_history, history_path)
else:
    model_fm_1 = tf.keras.models.load_model(disk_path + model_path)
    model_fm_1_history = joblib.load(disk_path + history_path)

Epoch 1/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 6s/step - loss: nan - mae: nan - mape: nan - mse: nan - rmse: nan - val_loss: nan - val_mae: nan - val_mape: nan - val_mse: nan - val_rmse: nan - learning_rate: 0.0300
Epoch 2/2
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: nan - mae: nan - mape: nan - mse: nan - rmse: nan

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: nan - mae: nan - mape: nan - mse: nan - rmse: nan - val_loss: nan - val_mae: nan - val_mape: nan - val_mse: nan - val_rmse: nan - learning_rate: 0.0297
Restoring model weights from the end of the best epoch: 1.


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


## 3. FM 2-3

### 3.1 Подготовка данных

In [16]:
df = generate_optimal_features_cu(df)
X_fm_2_3, y_fm_2_3 = make_dataset_cu(df)

print(f"X_fm_2_3.shape = {X_fm_2_3.shape}\n")
X_fm_2_3.sample(2)

X_fm_2_3.shape = (20422, 9)



,Mass,Dens,Cu_F,Ni_F,Cu_C,Ni_C,Cu_T,Ni_T,fm
19967,713.210815,1.367016,4.8970,2.9813,19.5016,2.7075,0.3371,3.0132,0
25342,1521.415881,1.352203,11.3184,2.6060,13.7361,2.8648,1.2078,5.1336,1


In [17]:
X_fm_2_3.describe()

,Mass,Dens,Cu_F,Ni_F,Cu_C,Ni_C,Cu_T,Ni_T,fm
count,20422.000000,20422.000000,19805.00000,19805.000000,18704.000000,18704.000000,19665.000000,19665.000000,20422.000000
mean,1021.262185,1.363186,9.15004,2.934163,16.306685,2.732193,0.735428,3.679156,0.474782
std,223.644084,0.110689,4.79152,0.253134,1.725598,0.269249,0.487549,0.780219,0.499376
min,0.000000,0.000000,3.55280,1.813000,10.259600,1.648600,0.247400,1.793000,0.000000
25%,832.558975,1.314441,4.76660,2.790600,14.886475,2.549900,0.320900,3.017400,0.000000
50%,920.675781,1.382080,5.42610,2.963600,16.439650,2.717600,0.389300,3.351300,0.000000
75%,1215.316711,1.433875,13.91530,3.057900,17.529325,2.921125,1.179100,4.371800,1.000000
max,1742.128578,1.566742,20.95740,5.454900,20.496300,4.523500,4.124000,6.991200,1.000000


In [18]:
print(f"y_fm_2_3.shape = {y_fm_2_3.shape}\n")
y_fm_2_3.sample(2)

y_fm_2_3.shape = (20422, 2)



,Cu_T_min,Cu_T_max
9520,0.8,1.5
13825,0.6,1.8


In [19]:
y_fm_2_3.describe()

,Cu_T_min,Cu_T_max
count,20422.000000,20422.000000
mean,0.514790,0.768686
std,0.285692,0.529938
min,0.230000,0.330000
25%,0.230000,0.420000
50%,0.360000,0.420000
75%,0.700000,0.900000
max,1.300000,2.000000


### 3.2. Модель

In [20]:
model_fm_2_3 = get_model_fm_2_3(X_fm_2_3)

In [21]:
model_fm_2_3.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Mass (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Dens (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Cu_F (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Ni_F (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Cu_C (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Ni_C (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Cu_T (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Ni_T (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Mass (Normalization) │ (None, 1)              │              3 │ Mass[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Dens (Normalization) │ (None, 1)              │              3 │ Dens[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Cu_F (Normalization) │ (None, 1)              │              3 │ Cu_F[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Ni_F (Normalization) │ (None, 1)              │              3 │ Ni_F[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Cu_C (Normalization) │ (None, 1)              │              3 │ Cu_C[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Ni_C (Normalization) │ (None, 1)              │              3 │ Ni_C[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Cu_T (Normalization) │ (None, 1)              │              3 │ Cu_T[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Ni_T (Normalization) │ (None, 1)              │              3 │ Ni_T[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ fm (InputLayer)           │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ all_features              │ (None, 9)              │              0 │ norm_Mass[0][0],       │
│ (Concatenate)             │                        │                │ norm_Dens[0][0],       │
│                           │                        │                │ norm_Cu_F[0][0],       │
│                           │                        │                │ norm_Ni_F[0][0],       │
│                      

 Total params: 906 (3.57 KB)

 Trainable params: 882 (3.45 KB)

 Non-trainable params: 24 (128.00 B)

### 3.3. Dataset

In [22]:
df2 = pd.concat([X_fm_2_3, y_fm_2_3], axis=1).reset_index()
N = int(len(df2)*0.15)

valid_ind= np.random.choice(len(df2), N, replace=False)
train_ind = list(set(df2.index) - set(valid_ind))

ds_train_fm_2_3 = get_tf_dataset_fm_2_3(df2.loc[train_ind, :])
ds_valid_fm_2_3 = get_tf_dataset_fm_2_3(df2.loc[valid_ind, :])

In [23]:
ds_train_fm_2_3.element_spec

({'Mass': TensorSpec(shape=(1,), dtype=tf.float64, name='Mass'),
  'Dens': TensorSpec(shape=(1,), dtype=tf.float64, name='Dens'),
  'Cu_F': TensorSpec(shape=(1,), dtype=tf.float64, name='Cu_F'),
  'Ni_F': TensorSpec(shape=(1,), dtype=tf.float64, name='Ni_F'),
  'Cu_C': TensorSpec(shape=(1,), dtype=tf.float64, name='Cu_C'),
  'Ni_C': TensorSpec(shape=(1,), dtype=tf.float64, name='Ni_C'),
  'Cu_T': TensorSpec(shape=(1,), dtype=tf.float64, name='Cu_T'),
  'Ni_T': TensorSpec(shape=(1,), dtype=tf.float64, name='Ni_T'),
  'fm': TensorSpec(shape=(1,), dtype=tf.int64, name='fm')},
 TensorSpec(shape=(2,), dtype=tf.float64, name='output'))

In [24]:
X, y = next(iter(ds_train_fm_2_3.take(1)))
model_fm_2_3(X)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.54577905, 0.37336388]], dtype=float32)>

### 3.4. Обучение модели

In [25]:
i = 1
model_path = f"models/model_fm_2_3_v{i}.keras"
history_path = f"models/model_fm_2_3_training_history_v{i}.joblib"

disk_path = "/content/drive/MyDrive/ML_projects/10_ts_optimisation/"
checkpoints_path = disk_path + f'data/06_model_output/initial_weights_{i}.weights.h5'

BATCH_SIZE = 128

regenerate = True
if regenerate:
    initial_weights = os.path.join(tempfile.mkdtemp(), checkpoints_path)

    model_fm_2_3 = get_model_fm_2_3(X_fm_2_3)
    model_fm_2_3.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.Huber(),
        metrics=METRICS
    )
    #model_fm_4_5_6.load_weights(initial_weights)

    n_epochs = 2
    steps_per_epoch= 2
    batch_size = BATCH_SIZE
    reduce_lr = tf.keras.callbacks.LearningRateScheduler(
        lambda epoch: 3e-2 * 0.99**epoch
    )
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=30,
        min_delta=1e-06,
        verbose=1,
        mode="min",
        restore_best_weights=True,
    )

    model_fm_2_3_history = model_fm_2_3.fit(
        ds_train_fm_2_3.batch(batch_size),
        epochs=n_epochs,
        steps_per_epoch=steps_per_epoch,
        validation_data=ds_valid_fm_2_3.batch(batch_size),
        batch_size=batch_size,
        verbose=1,
        callbacks=[reduce_lr, early_stopping],
        shuffle=True,
    )
    model_fm_2_3.save(model_path)
    joblib.dump(model_fm_2_3_history, history_path)
else:
    model_fm_2_3 = tf.keras.models.load_model(disk_path + model_path)
    model_fm_2_3_history = joblib.load(disk_path + history_path)

Epoch 1/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 7s/step - loss: nan - mae: nan - mape: nan - mse: nan - rmse: nan - val_loss: nan - val_mae: nan - val_mape: nan - val_mse: nan - val_rmse: nan - learning_rate: 0.0300
Epoch 2/2
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: nan - mae: nan - mape: nan - mse: nan - rmse: nan

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: nan - mae: nan - mape: nan - mse: nan - rmse: nan - val_loss: nan - val_mae: nan - val_mape: nan - val_mse: nan - val_rmse: nan - learning_rate: 0.0297
Restoring model weights from the end of the best epoch: 1.


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


## 4. FM 4-6

### 4.1 Подготовка данных

In [26]:
df = generate_optimal_features_ni(df)
X_fm_4_5_6, y_fm_4_5_6 = make_dataset_ni(df)

print(f"X_fm_4_5_6.shape = {X_fm_4_5_6.shape}\n")
X_fm_4_5_6.sample(2)

X_fm_4_5_6.shape = (35113, 8)



,Mass,Vol,Dens,Ni_F,Ni_C,Ni_T,Cu_F,fm
21131,863.039368,1599.8750,1.401477,2.68810,5.0275,0.7246,0.3196,0
8502,547.213257,1058.3125,1.384821,4.50685,6.5690,0.6773,0.0000,1


In [27]:
X_fm_4_5_6.describe()

,Mass,Vol,Dens,Ni_F,Ni_C,Ni_T,Cu_F,fm
count,35113.000000,35113.000000,35113.000000,34705.000000,35113.000000,35113.000000,34865.000000,35113.000000
mean,626.218867,1214.803424,1.377717,4.451208,6.345717,0.779096,0.203517,0.708740
std,203.778457,329.777029,0.037523,2.101145,1.634758,0.203509,0.249643,0.697216
min,0.000000,0.000000,0.000000,1.986900,3.739700,0.446300,0.000000,0.000000
25%,485.953857,993.375000,1.357727,2.885000,4.811700,0.675800,0.000000,0.000000
50%,551.674316,1061.625000,1.380666,4.221550,6.510800,0.706700,0.000000,1.000000
75%,831.504028,1566.218750,1.401477,4.636900,7.234300,0.749200,0.449200,1.000000
max,1263.258057,1980.687500,1.651951,10.403200,10.783400,1.858600,3.629900,2.000000


In [28]:
print(f"y_fm_4_5_6.shape = {y_fm_4_5_6.shape}\n")
y_fm_4_5_6.sample(2)

y_fm_4_5_6.shape = (35113, 4)



,Ni_C_min,Ni_C_max,Ni_T_min,Ni_T_max
10204,8.1,8.8,1.00,1.25
16092,4.0,5.2,0.55,0.80


In [29]:
y_fm_4_5_6.describe()

,Ni_C_min,Ni_C_max,Ni_T_min,Ni_T_max
count,35113.000000,35113.000000,35113.000000,35113.000000
mean,5.697982,6.426567,0.671118,0.847799
std,1.753353,1.494305,0.216682,0.236213
min,3.500000,4.200000,0.550000,0.700000
25%,4.000000,5.200000,0.550000,0.700000
50%,6.000000,6.800000,0.550000,0.780000
75%,6.800000,7.000000,0.680000,0.800000
max,9.800000,10.000000,1.550000,1.900000


### 4.2. Модель

In [30]:
model_fm_4_5_6 = get_model_fm_4_5_6(X_fm_4_5_6)

In [31]:
model_fm_4_5_6.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Mass (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Vol (InputLayer)          │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Dens (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Ni_F (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Ni_C (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Ni_T (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Cu_F (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ fm (InputLayer)           │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Mass (Normalization) │ (None, 1)              │              3 │ Mass[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Vol (Normalization)  │ (None, 1)              │              3 │ Vol[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Dens (Normalization) │ (None, 1)              │              3 │ Dens[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Ni_F (Normalization) │ (None, 1)              │              3 │ Ni_F[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Ni_C (Normalization) │ (None, 1)              │              3 │ Ni_C[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Ni_T (Normalization) │ (None, 1)              │              3 │ Ni_T[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ norm_Cu_F (Normalization) │ (None, 1)              │              3 │ Cu_F[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ intlookup_fm              │ (None, 4)              │              0 │ fm[0][0]               │
│ (IntegerLookup)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ all_features              │ (None, 11)             │              0 │ norm_Mass[0][0],       │
│ (Concatenate)             │                        │                │ norm_Vol[0][0],        │
│                           │                        │                │ norm_Dens[0][0],       │
│                           │                        │                │ norm_Ni_F[0][0],       │
│                           │                        │                │ norm_Ni_C[0][0],       │
│                      

 Total params: 1,001 (3.94 KB)

 Trainable params: 980 (3.83 KB)

 Non-trainable params: 21 (112.00 B)

### 4.3. Dataset

In [32]:
df3 = pd.concat([X_fm_4_5_6, y_fm_4_5_6], axis=1).reset_index()
N = int(len(df3)*0.15)

valid_ind= np.random.choice(len(df3), N, replace=False)
train_ind = list(set(df3.index) - set(valid_ind))

ds_train_fm_4_5_6 = get_tf_dataset_fm_4_5_6(df3.loc[train_ind, :])
ds_valid_fm_4_5_6 = get_tf_dataset_fm_4_5_6(df3.loc[valid_ind, :])

In [33]:
ds_train_fm_4_5_6.element_spec

({'Mass': TensorSpec(shape=(1,), dtype=tf.float64, name='Mass'),
  'Vol': TensorSpec(shape=(1,), dtype=tf.float64, name='Vol'),
  'Dens': TensorSpec(shape=(1,), dtype=tf.float64, name='Dens'),
  'Ni_F': TensorSpec(shape=(1,), dtype=tf.float64, name='Ni_F'),
  'Ni_C': TensorSpec(shape=(1,), dtype=tf.float64, name='Ni_C'),
  'Ni_T': TensorSpec(shape=(1,), dtype=tf.float64, name='Ni_T'),
  'Cu_F': TensorSpec(shape=(1,), dtype=tf.float64, name='Cu_F'),
  'fm': TensorSpec(shape=(1,), dtype=tf.int64, name='fm')},
 TensorSpec(shape=(4,), dtype=tf.float64, name='output'))

### 4.4. Обучение модели

In [34]:
METRICS = [
    tf.keras.metrics.MeanAbsoluteError(name="mae"),
    tf.keras.metrics.MeanAbsolutePercentageError(name="mape"),
    tf.keras.metrics.MeanSquaredError(name="mse"),
    tf.keras.metrics.RootMeanSquaredError(name="rmse")
]

In [35]:
i = 1
model_path = f"models/model_fm_4_5_6_v{i}.keras"
history_path = f"models/model_fm_4_5_6_training_history_v{i}.joblib"

disk_path = "/content/drive/MyDrive/ML_projects/10_ts_optimisation/"
checkpoints_path = disk_path + f'data/06_model_output/initial_weights_{i}.weights.h5'

BATCH_SIZE = 128

regenerate = True
if regenerate:
    initial_weights = os.path.join(tempfile.mkdtemp(), checkpoints_path)

    model_fm_4_5_6 = get_model_fm_4_5_6(X_fm_4_5_6)
    model_fm_4_5_6.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.Huber(),
        metrics=METRICS
    )
    #model_fm_4_5_6.load_weights(initial_weights)

    n_epochs = 2
    steps_per_epoch= 2
    batch_size = BATCH_SIZE
    reduce_lr = tf.keras.callbacks.LearningRateScheduler(
        lambda epoch: 3e-2 * 0.99**epoch
    )
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=30,
        min_delta=1e-06,
        verbose=1,
        mode="min",
        restore_best_weights=True,
    )

    model_fm_4_5_6_history = model_fm_4_5_6.fit(
        ds_train_fm_4_5_6.batch(batch_size),
        epochs=n_epochs,
        steps_per_epoch=steps_per_epoch,
        validation_data=ds_valid_fm_4_5_6.batch(batch_size),
        batch_size=batch_size,
        verbose=1,
        callbacks=[reduce_lr, early_stopping],
        shuffle=True,
    )
    model_fm_4_5_6.save(model_path)
    joblib.dump(model_fm_4_5_6_history, history_path)
else:
    model_fm_4_5_6 = tf.keras.models.load_model(disk_path + model_path)
    model_fm_4_5_6_history = joblib.load(disk_path + history_path)

Epoch 1/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 11s/step - loss: nan - mae: nan - mape: nan - mse: nan - rmse: nan - val_loss: nan - val_mae: nan - val_mape: nan - val_mse: nan - val_rmse: nan - learning_rate: 0.0300
Epoch 2/2
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: nan - mae: nan - mape: nan - mse: nan - rmse: nan

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2/2 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/step - loss: nan - mae: nan - mape: nan - mse: nan - rmse: nan - val_loss: nan - val_mae: nan - val_mape: nan - val_mse: nan - val_rmse: nan - learning_rate: 0.0297
Restoring model weights from the end of the best epoch: 1.
